<a href="https://colab.research.google.com/github/AlokDhanush/article_summarizer/blob/main/article_summarizer_bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset["train"].select(range(1000))
val_dataset = dataset["validation"].select(range(100))
test_dataset = dataset["test"].select(range(100))

In [ ]:
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
def preprocess_data(examples):
    inputs = [article for article in examples["article"]]
    targets = [summary for summary in examples["highlights"]]

    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
tokenized_dataset_train = train_dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights"])
tokenized_dataset_test = test_dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights"])
tokenized_dataset_val = val_dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tf_train_dataset = tokenized_dataset_train.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator
)

tf_val_dataset = tokenized_dataset_test.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=2,
    collate_fn=data_collator
)

tf_val_dataset = tokenized_dataset_val.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=2,
    collate_fn=data_collator
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=3)

Epoch 1/3
500/500 [==============================] - 197s 211ms/step - loss: 2.8153 - val_loss: 2.3659
Epoch 2/3
500/500 [==============================] - 97s 193ms/step - loss: 2.2030 - val_loss: 2.3287
Epoch 3/3
500/500 [==============================] - 97s 194ms/step - loss: 1.8367 - val_loss: 2.4506


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f07d1b87d4af5ab6e069aaf1ee5cdfb472c8d9815c3efaf7032c45e649bbae1b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Specify the directory where you want to save the model and tokenizer
save_directory = "./bart_cnn_dailymail_model"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart_cnn_dailymail_model/tokenizer_config.json',
 './bart_cnn_dailymail_model/special_tokens_map.json',
 './bart_cnn_dailymail_model/vocab.json',
 './bart_cnn_dailymail_model/merges.txt',
 './bart_cnn_dailymail_model/added_tokens.json',
 './bart_cnn_dailymail_model/tokenizer.json')

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at ./bart_cnn_dailymail_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
test_dataset[0]

{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

In [ ]:
# Save as a TensorFlow SavedModel
loaded_model.save("./bart_cnn_dailymail_savedmodel", save_format="tf")

In [ ]:
def generate_summary(article):
    inputs = loaded_tokenizer(article, return_tensors="tf", max_length=512, truncation=True, padding="max_length")

    summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return generated_summary

In [ ]:
article_text = "(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed 'in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014.' Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony, said it was a move toward greater justice. 'As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice,' he said, according to an ICC news release. 'Indeed, today brings us closer to our shared goals of justice and peace.' Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. 'As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly,' she said. Rights group Human Rights Watch welcomed the development. 'Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court\'s treaty should speak out to welcome its membership,' said Balkees Jarrah, international justice counsel for the group. 'What\'s objectionable is the attempts to undermine international justice, not Palestine\'s decision to join a treaty to which over 100 countries around the world are members.' In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it 'strongly' disagreed with the court\'s decision. 'As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC,' the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. 'We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace,' it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as 'Palestine.' While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would 'conduct its analysis in full independence and impartiality.' The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN\'s Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report."

print("Generated Summary: ", generate_summary(article_text))

Generated Summary:  Palestinian Authority officially becomes 123rd member of the International Criminal Court.
The court has jurisdiction over alleged crimes committed in occupied Palestinian territories.
Israel and the United States opposed Palestinians' efforts to join the body.


In [ ]:
article_text = "In recent years, the concept of sustainable living has gained significant traction among individuals, communities, and governments worldwide. As climate change becomes an increasingly pressing issue, the need to adopt sustainable practices has never been more urgent. Sustainable living refers to a lifestyle that attempts to reduce an individual's or society's use of the Earth's natural resources. It emphasizes minimizing waste, conserving energy, and protecting the environment to ensure a healthy planet for future generations. One of the primary aspects of sustainable living is reducing waste. Every year, millions of tons of waste end up in landfills, contributing to environmental degradation and pollution. By adopting practices such as recycling, composting, and reducing single-use plastics, individuals can significantly decrease their waste output. Communities are also implementing recycling programs and initiatives to educate residents on responsible waste management. Energy conservation is another critical component of sustainable living. Traditional energy sources, such as fossil fuels, are finite and contribute to greenhouse gas emissions. By shifting towards renewable energy sources like solar, wind, and hydroelectric power, individuals and communities can reduce their carbon footprint. Simple actions, such as using energy-efficient appliances, turning off lights when not in use, and utilizing public transportation, can lead to significant energy savings. Moreover, sustainable living promotes the conservation of biodiversity. The loss of biodiversity can have devastating effects on ecosystems and the services they provide, such as clean air and water. By supporting local agriculture and purchasing organic products, consumers can help protect wildlife habitats and reduce the environmental impact of food production. Sustainable farming practices, including crop rotation and organic farming, contribute to healthier ecosystems and food systems. Education and awareness are crucial in fostering a culture of sustainability. Schools, organizations, and community groups are increasingly offering programs and workshops to teach individuals about the importance of sustainable practices. By empowering people with knowledge and resources, we can inspire collective action toward a more sustainable future. In conclusion, sustainable living is essential for preserving our planet and ensuring that future generations inherit a healthy environment. By making conscious choices in our daily lives, we can reduce waste, conserve energy, and protect biodiversity. Embracing sustainability not only benefits the environment but also enhances our quality of life and fosters a sense of community. Together, we can make a significant impact and create a more sustainable world."

print("Generated Summary: ", generate_summary(article_text))

Generated Summary:  Sustainable living emphasizes minimizing waste, conserving energy, protecting the environment.
The concept of sustainable living has gained significant traction among individuals, communities, and governments.
Consumers can reduce waste, conserve energy, protect biodiversity.


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

def evaluate_rouge1(model, tokenizer, dataset):
    rouge1_scores = []

    for example in dataset:

        inputs = tokenizer(example["article"], return_tensors="tf", truncation=True, padding="max_length", max_length=128)
        summary_ids = model.generate(inputs["input_ids"], max_length=32, num_beams=4, early_stopping=True)


        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        reference_summary = example["highlights"]


        scores = scorer.score(reference_summary, generated_summary)
        rouge1_scores.append(scores['rouge1'].fmeasure)  # Store ROUGE-1 scores

    avg_rouge1_score = np.mean(rouge1_scores)
    return avg_rouge1_score


test_dataset_small = test_dataset.select(range(10))
rouge1_score = evaluate_rouge1(model, tokenizer, test_dataset_small)

print("Average ROUGE-1 Score:", rouge1_score)

Average ROUGE-1 Score: 0.3490119951049032
